# Tertiaire

In [2]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [10]:
# load the data and select only the columns we need
df = pd.read_csv('notebooks/rapprochements/DPE/dpe-v2-logements-existants.csv', sep=',', usecols=['N°DPE', 'Date_réception_DPE', 'Identifiant__BAN'])

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
df.shape

(8018355, 3)

In [6]:
# create 100 subfiles in the dpe_existant folder, using the number of rows in the dataframe
for i in range(0, 100):
    df.iloc[i*df.shape[0]//100:(i+1)*df.shape[0]//100].to_csv(f'notebooks/rapprochements/DPE/dpe_existants/dpe-existant-{i}.csv', index=False)


In [3]:
import os
from batid.models import Building
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from concurrent.futures import ThreadPoolExecutor

def get_buildings_from_ban_id(ban_id):
    buildings = Building.objects.filter(addresses_read_only__id=ban_id).values('rnb_id')
    return [b['rnb_id'] for b in buildings]


def process_file(i):
    # if the result file exists, skip i
    print(f'Processing file {i}')
    if not os.path.exists(f'notebooks/rapprochements/DPE/dpe_existants_results/dpe-existant-{i}-result.csv'):
        df = pd.read_csv(f'notebooks/rapprochements/DPE/dpe_existants/dpe-existant-{i}.csv', sep=',')
        df['rnb_id_from_ban_id'] = df['Identifiant__BAN'].apply(get_buildings_from_ban_id)
        df.to_csv(f'notebooks/rapprochements/DPE/dpe_existants_results/dpe-existant-{i}-result.csv', index=False)

with ThreadPoolExecutor(max_workers=10) as executor:
    executor.map(process_file, range(100))


Processing file 0
Processing file 1
Processing file 2
Processing file 3
Processing file 4
Processing file 5
Processing file 6
Processing file 7
Processing file 8
Processing file 9
Processing file 10
Processing file 11
Processing file 12
Processing file 13
Processing file 14
Processing file 15
Processing file 16
Processing file 17
Processing file 18
Processing file 19
Processing file 20
Processing file 21
Processing file 22
Processing file 23
Processing file 24
Processing file 25
Processing file 26
Processing file 27
Processing file 28
Processing file 29
Processing file 30
Processing file 31
Processing file 32
Processing file 33
Processing file 34
Processing file 35
Processing file 36
Processing file 37
Processing file 38
Processing file 39
Processing file 40
Processing file 41
Processing file 42
Processing file 43
Processing file 44
Processing file 45
Processing file 46
Processing file 47
Processing file 48
Processing file 49
Processing file 50
Processing file 51
Processing file 52
Pro

In [2]:
# concactenate all the results files
df = pd.concat([pd.read_csv(f'notebooks/rapprochements/DPE/dpe_existants_results/dpe-existant-{i}-result.csv', sep=',') for i in range(100)])
#df.to_csv('notebooks/rapprochements/DPE/dpe_existants_results/dpe-existant-all-results.csv', index=False)

In [3]:
df.shape

(8018355, 4)

In [4]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from concurrent.futures import ThreadPoolExecutor
# split the df dataframe into 100 sub data frames
# df_sample = df.head(10000)
dfs = [df.iloc[i*df.shape[0]//100:(i+1) * df.shape[0]//100] for i in range(100)]

# create a function to remove small surfaces
def remove_small_surfaces(rnb_ids):
    results = []
    for rnb_id in rnb_ids:
        raw_sql = f"SELECT id, ST_AREA(shape::geography) FROM batid_building WHERE rnb_id = '{rnb_id}'"
        total_surface = Building.objects.raw(raw_sql)[0].st_area
        if total_surface > 25:
            results.append(rnb_id)
    return results

# multi thread apply the function to the dataframes and concatenate the results
def rnb_id_filtered(df):
    df_copy = df.copy()
    df_copy['rnb_id_filtered'] = df['rnb_id_from_ban_id'].apply(eval).apply(remove_small_surfaces)
    return df_copy

with ThreadPoolExecutor(max_workers=20) as executor:
    results = executor.map(rnb_id_filtered, dfs)
    df_filtered = pd.concat(results)
    df_filtered.to_csv('notebooks/rapprochements/DPE/dpe_existants_results/dpe-existants-results-filtered.csv', index=False)


In [3]:
df_results_filtered = pd.read_csv('notebooks/rapprochements/DPE/dpe_existants_results/dpe-existants-results-filtered.csv', sep=',')

Pourcentage de match 1-1 avant filtre des petites surfaces

In [4]:
df_results_filtered['rnb_id_from_ban_id'].apply(eval).apply(len).value_counts()[1] / df_results_filtered.shape[0] * 100

59.2808001142379

Pourcentage de match 1-1 APRES filtre des petites surfaces

In [5]:
df_results_filtered['rnb_id_filtered'].apply(eval).apply(len).value_counts()[1] / df_results_filtered.shape[0] * 100

60.471168961713474

Ecriture de rapprochement fichier final pour envoi 

In [8]:
df_results_filtered[['N°DPE', 'Date_réception_DPE', 'rnb_id_filtered']].to_csv('notebooks/rapprochements/DPE/dpe_existants_results/dpe-v2-logements-existants-rapprochement-final-results.csv', index=False, sep=',')

: 